In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import libraries 📚

In [ ]:
from IPython.display import clear_output

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import datetime
import tensorflow as tf
import keras
import itertools
import matplotlib
import matplotlib.pyplot as plt
#from pylab import rcParams
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, BatchNormalization, Input, concatenate, Reshape
from keras import layers, Input, Model
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import models, regularizers, optimizers
from keras.models import Sequential, save_model
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
from sklearn.utils import class_weight
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from sklearn.metrics import log_loss
import tensorflow_addons as tfa
from sklearn.model_selection import StratifiedKFold
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import Convolution1D
from keras.layers.convolutional import MaxPooling1D
import pandas as pd
import random
from tensorflow.keras.layers import Dense, Conv2DTranspose, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
import math
from PIL import Image
import plotly.express as px
import math
print('Done..')

# Reading the CSV File 📁

In [ ]:
train=pd.read_csv('../input/emnist/emnist-digits-train.csv')
test=pd.read_csv('../input/emnist/emnist-digits-test.csv')
a=['label']
c=0
for i in range(784):
    a.append('pixel'+str(i))
train.columns=a
test.columns=a    

In [ ]:
df=pd.read_csv('../input/digit-recognizer/train.csv')
X=df.drop('label', axis=1)
y=df.label
df.head()

In [ ]:
x_train=df.drop('label', axis=1)
Y_train=df.label
x_train=x_train.values
Y_train=Y_train.values

In [ ]:
import matplotlib.pyplot as plt
print('Some examples from training set...')
# look at some of the digits from train_X
plt.figure(figsize=(15,6))
for i in range(40):  
    plt.subplot(4, 10, i+1)
    plt.imshow(x_train[i].reshape((28,28)),cmap=plt.cm.binary)
    plt.title("label=%d" % Y_train[i],y=0.9)
    plt.axis('off')
plt.subplots_adjust(wspace=0.3, hspace=-0.1)
plt.show()

In [ ]:
x_train=train.drop('label', axis=1)
Y_train=train.label
x_train=x_train.values
Y_train=Y_train.values

In [ ]:
import matplotlib.pyplot as plt
print('Some examples from additional dataset...')
plt.figure(figsize=(15,6))
for i in range(40):  
    plt.subplot(4, 10, i+1)
    plt.imshow(x_train[i].reshape((28,28)),cmap=plt.cm.binary)
    plt.title("label=%d" % Y_train[i],y=0.9)
    plt.axis('off')
plt.subplots_adjust(wspace=0.3, hspace=-0.1)
plt.show()

In [ ]:
del x_train
del Y_train

In [ ]:
df=pd.concat([df,train,test])
df.reset_index()

In [ ]:
train=pd.read_csv('../input/emnist/emnist-mnist-train.csv')
test=pd.read_csv('../input/emnist/emnist-mnist-test.csv')
train.columns=a
test.columns=a  

In [ ]:
df=pd.concat([df,train,test])
df.reset_index()

In [ ]:
df = df.sample(frac = 1)
X=df.drop('label', axis=1)
y=df.label
y=pd.get_dummies(y)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.15, random_state=42)
test_df=pd.read_csv('../input/digit-recognizer/test.csv')
X_test = test_df

In [ ]:
del X
del y
del df

# CNN model 👻

In [ ]:
def create_model(num_column):
    model = Sequential()
    model.add(tf.keras.layers.Reshape((28, 28, 1), input_shape=(784,)))
    model.add(BatchNormalization())
    model.add(layers.Conv2D(64, (3,3), activation='relu', input_shape=(28,28,1)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(layers.Conv2D(256, (3, 3), activation='relu'))
    model.add(BatchNormalization()) 
    model.add(Dropout(0.4))
    
    model.add(Flatten())
    model.add(Dense(units=200, activation='relu'))
    model.add(Dense(units=100, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(units=10, activation='softmax'))
    model.compile(
    optimizer='adam',loss='categorical_crossentropy',metrics=tf.keras.metrics.CategoricalAccuracy())
    return model

In [ ]:
# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

# instantiating the model in the strategy scope creates the model on the TPU
with tpu_strategy.scope():
    model=create_model(784)
    #model.compile(optimizer=tf.keras.optimizers.Adam(), loss='mse', metrics=['mae'])
model.summary()

In [ ]:
es = EarlyStopping(monitor='val_categorical_accuracy', mode='max', verbose=1, patience=175)
mc = ModelCheckpoint('best_model.h5', monitor='val_categorical_accuracy', mode='max', verbose=1, save_best_only=True, patience=175)

In [ ]:
history=model.fit(X_train, y_train, epochs=1000, validation_data=(X_validation, y_validation), batch_size=20480, verbose=1,callbacks=[es,mc])

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
 model=tf.keras.models.load_model('./best_model.h5')

In [ ]:
predi=model.predict(X_test)

In [ ]:
df=pd.DataFrame(predi)
df=df.apply(lambda x: x == x.max(), axis=1).astype(int)
df['ImageId']=df.index
df.set_index('ImageId',inplace=True)
df=df[df==1].stack().reset_index().drop(0,1)
df['Label']=df['level_1']
df=df.drop('level_1', axis=1)
df['ImageId']=df['ImageId']+1

In [ ]:
df.head()

In [ ]:
import gzip
df.to_csv('sub.gz', index=False, compression='gzip')